In [25]:
import cv2
import numpy as np
from pypylon import pylon
from time import time
import os

fileloc = r"C:\Users\tantirim\Documents\eyeTrack"
if not os.path.exists(fileloc):
    os.makedirs(fileloc)

camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
print("Using device ", camera.GetDeviceInfo().GetModelName())

def imgprocess(frame, r, thr, resize):
    frame = frame.copy()
    roi = frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    roi = cv2.resize(roi, dsize=None, fx=resize, fy=resize)
    rows, cols, _ = roi.shape
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_roi = cv2.medianBlur(gray_roi,11) 
    thr2 = np.amin(gray_roi)+thr
    _, threshold = cv2.threshold(gray_roi, thr2, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    
    for cnt in contours:      
        (x2,y2),radius = cv2.minEnclosingCircle(cnt)
        cv2.circle(roi, (int(x2),int(y2)), int(radius), (0, 0, 255), 1)
        cv2.line(roi, (int(x2), 0), (int(x2), rows), (0, 255, 0), 1)
        cv2.line(roi, (0, int(y2)), (cols, int(y2)), (0, 255, 0), 1)
        break    
    
    return roi

Using device  daA1280-54um


In [26]:
# Grab single frame to specify ROI

camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly) 
converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

if __name__ == '__main__' :
    grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
    if grabResult.GrabSucceeded():
        image = converter.Convert(grabResult)
        frame = image.GetArray()
        rows, cols, _ = frame.shape
        r = cv2.selectROI(frame)
        fps_cv2 = camera.ResultingFrameRate.GetValue();
        print("Frame rate is ", fps_cv2)
        cv2.waitKey(0)
    else:
        print("Camera didnt grab")
    grabResult.Release()
    
camera.StopGrabbing()        
cv2.destroyAllWindows()

Frame rate is  33.33444448148271


In [27]:
# Grabing frames continusely

filename = "basler_n0003_lep" #saveas name, will overwrite if unchanged
thr = 10 # adjust threshold
resize = 3

camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly) 
converter = pylon.ImageFormatConverter()
converter.OutputPixelFormat = pylon.PixelType_BGR8packed
converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
out_raw = cv2.VideoWriter(fileloc + "\\" + filename + "_raw.avi",fourcc, 
                      float(fps_cv2), (int(cols),int(rows)))

start = time()
t0 = 0
while camera.IsGrabbing():
    grabResult = camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
    if grabResult.GrabSucceeded():        
        image = converter.Convert(grabResult)
        img = image.GetArray()
        roi = imgprocess(img, r, thr,resize)        

        d = round(camera.ResultingFrameRate.GetValue(),2);        
        now = round(float(time()-start),2)        
        ts = '{} fps, t = {} s'.format(d, now)        
        cv2.putText(roi, ts, (10,20), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255, 100, 0),1)
        out_raw.write(img)
        cv2.moveWindow("Roi", 2,100)
        cv2.moveWindow("title", 500,100)        
        cv2.imshow('title', img)
        cv2.imshow("Roi", roi)

        k = cv2.waitKey(1)
        if k == 27:
            break
    else:
        print("Camera didnt grab")
    grabResult.Release()
       
camera.StopGrabbing()
out_raw.release()
cv2.destroyAllWindows()

In [18]:
camera.StopGrabbing() # stop camera manually